In [5]:
import neo4j

host = "localhost"
username = "neo4j"
password = "1234qwer"

# driver = neo4j.GraphDatabase.driver("neo4j://100.27.33.222:7687",
#   auth=neo4j.basic_auth("neo4j", "price-oxygens-scores")
# )
driver = neo4j.GraphDatabase.driver(f'bolt://localhost:7687', auth=(username, password))
session = driver.session()

def create_graph(query):
    session.run(query)

Failed to write data to connection IPv4Address(('localhost', 7687)) (ResolvedIPv4Address(('127.0.0.1', 7687)))


In [11]:
import nltk
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

# Download required NLTK data
nltk.download('punkt')
nltk.download('stopwords')

def preprocess_text(text, remove_stopwords=True, stem=False, lemmatize=True):
    """
    Preprocess text by applying various NLP techniques
    """
    # Convert to lowercase
    text = text.lower()
    
    # Remove punctuation
    text = text.translate(str.maketrans("", "", string.punctuation))
    
    # Remove numbers
    text = ''.join(char for char in text if not char.isdigit())
    
    # Tokenization
    tokens = word_tokenize(text)
    
    # Remove stopwords
    if remove_stopwords:
        stop_words = set(stopwords.words('english'))
        tokens = [token for token in tokens if token not in stop_words]
    
    # Join tokens back into text
    processed_text = ' '.join(tokens)
    
    return processed_text

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/fahmisyaifudin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/fahmisyaifudin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
records, summary, key = driver.execute_query("""
MATCH (n:CORE|VERB|HARDWARE|USER|PLATFORM) RETURN n.name""")
origin_entities = []
for record in records:
    name = record['n.name']
    origin_entities.append(name)

In [13]:
clean_entities = [preprocess_text(entity.strip()) for entity in origin_entities]

In [14]:
import pandas as pd

df_combined = pd.DataFrame({
    'entity1': origin_entities,
    'entity2': clean_entities
})

In [15]:
for index, row in df_combined.iterrows():
    session.run("""MATCH (n:CORE|VERB|USER|PLATFORM|HARDWARE {name: $entity1}) SET n.name = $entity2""", entity1=row['entity1'], entity2=row['entity2'])

In [2]:

from sentence_transformers import SentenceTransformer
from transformers import BertTokenizer, BertModel

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")

def sentence_transformer_embedding(text):
    model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    embeddings = model.encode(text)
    return embeddings

def bert_embedding(text):
    encoded_input = tokenizer(text, return_tensors='pt')
    output = model(**encoded_input)
    pooler_output = output['pooler_output'][0]
    return pooler_output.detach().numpy()

/Users/fahmisyaifudin/miniconda3/envs/llm/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
records, summary, key = driver.execute_query("""
MATCH (n:CORE|VERB|USER|PLATFORM|HARDWARE) WHERE n.name = '' RETURN DISTINCT n.name""")
entities = []
for record in records:
    name = record['n.name']
    entities.append(name)

#embedding = sentence_transformer_embedding(entities)

In [21]:
for i in range(len(entities)):
    try:
        embedding = bert_embedding(name)
        session.run(f"""
        MATCH (n:CORE|VERB|USER|PLATFORM|HARDWARE {{name: '{name}'}})
        SET n.bert_embedding = $embedding
        """, embedding=embedding)
    except Exception as e:
        print(f"Error processing entity {name}: {str(e)}")

In [59]:
session.run(""" 
MATCH (source:HARDWARE)
RETURN gds.graph.project(
  'newHardwareGraph',
  source,
  null,
  {
    sourceNodeProperties: source { .sbert_embedding },
    targetNodeProperties: {}
  },
  { undirectedRelationshipTypes: ['*'] }
)
""")

In [60]:
import pandas as pd

entities = []
records, summary, key = driver.execute_query("""
CALL gds.knn.stream('newHardwareGraph', {
    topK: 5,
    nodeProperties: ['sbert_embedding'],
    randomSeed: 1337,
    concurrency: 1,
    sampleRate: 1.0,
    deltaThreshold: 0.0
})
YIELD node1, node2, similarity
RETURN gds.util.asNode(node1).name AS Req1, gds.util.asNode(node2).name AS Req2, similarity
ORDER BY similarity DESCENDING, Req1, Req2""")
for record in records:
    entities.append({"entity1": record['Req1'], "entity2": record['Req2'], "similarity": record['similarity']})
df_entity = pd.DataFrame(entities)

In [61]:
df_entity.to_csv("output/entity_resolution/hardware_1.csv", index=False)

In [62]:
df_entity = pd.read_csv("output/entity_resolution/hardware_1.csv")

In [63]:
for index, row in df_entity.iterrows():
    session.run("""
MATCH (old:HARDWARE {name: $entity1})
MATCH (new:HARDWARE {name: $entity2})

WITH old, new
MATCH (old)<-[:CONTAINS]-(target)
MERGE (new)<-[:CONTAINS]-(target)
                
WITH old, new
MATCH (old)-[:RELATED_TO]->(target)
MERGE (new)-[:RELATED_TO]->(target)

WITH old, new
MATCH (source)-[:RELATED_TO]->(old)
MERGE (source)-[:RELATED_TO]->(new)

WITH old, new
DETACH DELETE old
    """, entity1=row['entity1'], entity2=row['entity2'])

In [7]:
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

client = OpenAI()

def get_embedding(text, model="text-embedding-ada-002"):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding

In [ ]:
record, summary, key = driver.execute_query(f"""
    MATCH (n:CORE|VERB|USER|PLATFORM|HARDWARE)
    RETURN DISTINCT(name) as name
    """)

In [ ]:
for i in range(len(entities)):
    name = entities[i]
    session.run(f"""
    MATCH (n:CORE|VERB|USER|PLATFORM|HARDWARE {{name: '{name}'}})
    SET n.embedding = $embedding
    """, embedding=embedding[i])